# [作業目標]
- 使用 Day 17 剛學到的方法, 對較完整的資料生成離散化特徵
- 觀察上述離散化特徵, 對於目標值的預測有沒有幫助

# [作業重點]
- 仿照 Day 17 的語法, 將年齡資料 ('DAYS_BIRTH' 除以 365) 離散化
- 繪製上述的 "離散化標籤" 與目標值 ('TARGET') 的長條圖

In [ ]:
# 載入需要的套件
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 設定 data_path
dir_data = './data/'

### 之前做過的處理

In [ ]:
# 讀取資料檔
f_app_train = os.path.join(dir_data, 'application_train.csv')
app_train = pd.read_csv(f_app_train)
app_train.shape

In [ ]:
# 將只有兩種值的類別型欄位, 做 Label Encoder, 計算相關係數時讓這些欄位可以被包含在內
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# 檢查每一個 column
for col in app_train:
    if app_train[col].dtype == 'object':
        # 如果只有兩種值的類別型欄位
        if len(list(app_train[col].unique())) <= 2:
            # 就做 Label Encoder, 以加入相關係數檢查
            app_train[col] = le.fit_transform(app_train[col])            
print(app_train.shape)
app_train.head()

In [ ]:
# 受雇日數為異常值的資料, 另外設一個欄位記錄, 並將異常的日數轉成空值 (np.nan)
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

# 出生日數 (DAYS_BIRTH) 取絕對值 
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])

## 練習時間
參考 Day 17 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [ ]:
# 初始設定受雇日數的資料
app_train['DAYS_EMPLOYED'] = abs(app_train['DAYS_EMPLOYED']) #取絕對值
emp = pd.DataFrame(app_train['DAYS_EMPLOYED'])


In [ ]:
# 等寬劃分受雇日數
emp['equal_width_days_employed'] = pd.cut(emp['DAYS_EMPLOYED'], bins=10)

In [ ]:
emp['equal_width_days_employed'].value_counts(0)

In [ ]:
# 等頻劃分受雇日數
emp['equal_freq_days_employed'] = pd.qcut(emp['DAYS_EMPLOYED'], 10)

In [ ]:
emp['equal_freq_days_employed'].value_counts()

In [ ]:
emp['customized_emp_grp'] = pd.cut(emp['DAYS_EMPLOYED'], [0, 2000, 4000, 6000, 8000, 10000, 12000, 14000, 16000, 17912])

In [ ]:
emp['customized_emp_grp'].value_counts()

In [ ]:
# 將emp['customized_emp_grp']用seaborn做長條圖
import seaborn as sns
plt.figure(figsize=(9, 9))
a = sns.barplot(emp['customized_emp_grp'], app_train['TARGET'])
plt.xticks(rotation = 50)
a.set_title('customized_emp_grp to TARGET')

In [ ]:
# 將emp['customized_emp_grp']用seaborn做密度圖(KDE)
emp_group_sorted = emp['DAYS_EMPLOYED'].value_counts().sort_index().index

plt.figure(figsize=(8,6))
for i in range(len(emp_group_sorted)):
    if len(emp.loc[(emp['DAYS_EMPLOYED'] == emp_group_sorted[i]) & \
                              (app_train['TARGET'] == 0), "DAYS_EMPLOYED"])==0: continue
    sns.distplot(emp.loc[(emp['DAYS_EMPLOYED'] == emp_group_sorted[i]) & \
                              (app_train['TARGET'] == 0), "DAYS_EMPLOYED"], label = str(emp_group_sorted[i]))
plt.title('KDE with Days of Employed groups with Target==0')
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i in range(len(emp_group_sorted)):
    if len(emp.loc[(emp['DAYS_EMPLOYED'] == emp_group_sorted[i]) & \
                              (app_train['TARGET'] == 0), "DAYS_EMPLOYED"])==0: continue
    sns.distplot(emp.loc[(emp['DAYS_EMPLOYED'] == emp_group_sorted[i]) & \
                              (app_train['TARGET'] == 1), "DAYS_EMPLOYED"], label = str(emp_group_sorted[i]))
plt.title('KDE with Days of Employed groups with Target==1')
plt.legend()
plt.show()

C:\Anaconda\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Anaconda\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Anaconda\lib\site-packages\statsmodels\nonparametric\kde.py:487: RuntimeWarning: invalid value encountered in true_divide
  binned = fast_linbin(X, a, b, gridsize) / (delta * nobs)
C:\Anaconda\lib\site-packages\statsmodels\nonparametric\kdetools.py:34: RuntimeWarning: invalid value encountered in double_scalars
  FAC1 = 2*(np.pi*bw/RANGE)**2
